In [1]:
# Import our dependencies
import os 
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from matplotlib import pyplot as plt
import plotly.express as px
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import tensorflow as tf

In [2]:
#Import the dataset 
india_companies_df = pd.read_csv('Resources/related_databases/India-startup-funding-2021-full.csv')
# india_companies_df = companies_df.drop(['Portfolio Exits'],axis = 1)
india_companies_df.head(10)

,Month,Company/Brand,Founded,Headquarters,Sector,What it does,Founder/s,Investor/s,Amount ($),Stage
0,December,CollegeDekho,2015.0,Gurgaon,E-learning,"Collegedekho.com is Student’s Partner, Friend ...",Ruchir Arora,"Disrupt ADQ, QIC",35000000,Series B
1,December,BOX8,2012.0,Mumbai,Food & Beverages,India's Largest Desi Meals Brand,"Anshul Gupta, Amit Raj",Tiger Global,40000000,NaN
2,December,Simpl,2015.0,Bangalore,Consumer Services,Simpl empowers merchants to build trusted rela...,"Nitya, Chaitra Chidanand","Valar Ventures, IA Ventures",40000000,Series B
3,December,8i Ventures,2018.0,Mumbai,Venture Capital & Private Equity,8i is a Mumbai & Bangalore based early stage f...,Vikram Chachra,NaN,50000000,NaN
4,December,PayGlocal,2021.0,Bangalore,Financial Services,PayGlocal is a FinTech solving for global paym...,"Prachi Dharani, Rohit Sukhija, Yogesh Lokhande",Sequoia Capital India,4900000,Series A
5,December,Curefit,2016.0,Bangalore,"Health, Wellness & Fitness",Tata Digital-backed Curefit,"Mukesh Bansal, Ankit",Zomato,145000000,NaN
6,December,Pocket FM,2018.0,Bangalore,OTT,Pocket FM lets users discover and enjoy the gr...,"Rohan, Nishanth Srinivas, Prateek Dixit","Lightspeed, Times Group, Tanglin Venture Partners",22000000,Series B
7,December,CHARGE+ZONE,2018.0,Vadodara,Automotive,CHARGE+ZONE is a tech-driven EV Charging infra...,Kartikey Hariyani,Venture Catalysts,10000000,NaN
8,December,PlanetSpark,2017.0,Gurugram,Education Management,PlanetSpark is building the World Leader in Co...,"Maneesh Dhooper, Kunal Malik","Binny Bansal, Deep Kalra, Dr Ashish Gupta, Gok...",13000000,Series B
9,December,LenDenClub,2015.0,Mumbai,Financial Services,P2P lending platform delivering high returns o...,"Bhavin Patel, Dipesh Karki","Tuscan Ventures, Ohm Stock Brokers, Artha Vent...",10000000,Series A


In [3]:
india_companies_df['Stage'].value_counts()

Seed             246
Pre-series A     148
Series A         126
Series B          49
Pre-seed          46
Series C          45
Debt              27
Series D          21
Series E          18
Pre-series B      10
Pre-Series A       9
Series F           6
Pre-series A1      4
Pre-series         2
1200000            2
Series H           2
300000             2
Series G           2
Bridge             2
Series I           1
Pre-Seed           1
Seed+              1
Series F2          1
Series A+          1
Series A2          1
Early seed         1
Series D1          1
6000000            1
1000000            1
Seies A            1
Series B3          1
PE                 1
Series F1          1
Name: Stage, dtype: int64

In [4]:
india_companies_df['Sector'].value_counts()

FinTech                              121
EdTech                               102
Financial Services                    61
Food & Beverages                      37
Information Technology & Services     36
                                    ... 
Environmental Services                 1
Merchandise                            1
Facilities Services                    1
D2C Business                           1
Fashion & Lifestyle                    1
Name: Sector, Length: 261, dtype: int64

In [5]:
india_companies_df['Month'].value_counts()

September    147
July         133
August       125
June         122
October      121
December     103
March         92
April         87
January       82
November      73
May           62
February      62
Name: Month, dtype: int64

In [6]:

india_companies_df = india_companies_df[india_companies_df['Founded']>= 2000]
india_companies_df = india_companies_df[india_companies_df['Amount ($)']!= 0]

india_companies_df['Amount ($)'].value_counts()

Undisclosed    133
1000000         85
2000000         59
3000000         47
10000000        42
              ... 
5400000          1
6750000          1
5700000          1
600000000        1
720000           1
Name: Amount ($), Length: 205, dtype: int64

In [7]:
india_companies_df = india_companies_df.drop(columns = ['Founder/s','What it does'])

In [8]:
for column in india_companies_df.columns:
    print (f"Column {column} has {india_companies_df[column].isnull().sum()} null values")



Column Month has 0 null values
Column Company/Brand has 0 null values
Column Founded has 0 null values
Column Headquarters has 1 null values
Column Sector has 0 null values
Column Investor/s has 59 null values
Column Amount ($) has 5 null values
Column Stage has 419 null values


In [9]:
#Replace NaN's

india_companies_df["Stage"] = india_companies_df["Stage"].replace(np.nan,"Other")
india_companies_df["Headquarters"] = india_companies_df["Headquarters"].replace(np.nan,"Other")
india_companies_df["Investor/s"] = india_companies_df["Investor/s"].replace(np.nan,"Other")
# india_companies_df["Founder/s"] = india_companies_df["Founder/s"].replace(np.nan,"Other")


# #Verify binning was successful 
# india_companies_df["Stage"].value_counts()

In [10]:
india_companies_df.set_index('Company/Brand',inplace = True)
india_companies_df.head(10)

,Month,Founded,Headquarters,Sector,Investor/s,Amount ($),Stage
Company/Brand,,,,,,,
CollegeDekho,December,2015.0,Gurgaon,E-learning,"Disrupt ADQ, QIC",35000000,Series B
BOX8,December,2012.0,Mumbai,Food & Beverages,Tiger Global,40000000,Other
Simpl,December,2015.0,Bangalore,Consumer Services,"Valar Ventures, IA Ventures",40000000,Series B
8i Ventures,December,2018.0,Mumbai,Venture Capital & Private Equity,Other,50000000,Other
PayGlocal,December,2021.0,Bangalore,Financial Services,Sequoia Capital India,4900000,Series A
Curefit,December,2016.0,Bangalore,"Health, Wellness & Fitness",Zomato,145000000,Other
Pocket FM,December,2018.0,Bangalore,OTT,"Lightspeed, Times Group, Tanglin Venture Partners",22000000,Series B
CHARGE+ZONE,December,2018.0,Vadodara,Automotive,Venture Catalysts,10000000,Other
PlanetSpark,December,2017.0,Gurugram,Education Management,"Binny Bansal, Deep Kalra, Dr Ashish Gupta, Gok...",13000000,Series B


In [11]:
india_encoded = pd.get_dummies(india_companies_df,columns = ['Month','Headquarters','Sector','Investor/s','Stage'])
india_encoded.head()

,Founded,Amount ($),Month_April,Month_August,Month_December,Month_February,Month_January,Month_July,Month_June,Month_March,...,Stage_Series C,Stage_Series D,Stage_Series D1,Stage_Series E,Stage_Series F,Stage_Series F1,Stage_Series F2,Stage_Series G,Stage_Series H,Stage_Series I
Company/Brand,,,,,,,,,,,,,,,,,,,,,
CollegeDekho,2015.0,35000000,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BOX8,2012.0,40000000,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Simpl,2015.0,40000000,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8i Ventures,2018.0,50000000,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PayGlocal,2021.0,4900000,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Months= india_encoded.loc[(india_encoded["Month_January","Month_February","Month_March","Month_April",
#                                          "Month_May","Month_June","Month_July","Month_August","Month_September",
#                                         "Month_October","Month_November","Month_December"])]
